In [2]:
import openai
from dotenv import dotenv_values
import pandas as pd
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [3]:
config = dotenv_values(".env")
openai.api_key = config["API"]

In [5]:
#import data set from https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots
dataset_path = "./wiki_movie_plots.csv"
source_df = pd.read_csv(dataset_path)

In [13]:
movies = source_df[source_df["Origin/Ethnicity"]=="American"].sort_values("Release Year", ascending=False).head(5000)

In [3]:
response = openai.Embedding.create(
    input=" ",
    model="text-embedding-ada-002"
)

In [ ]:
response["data"][0]["embedding"]